In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

import os

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.metrics import classification_report
import pickle

from SistemaHierarquico import Sistema_Hierarquico_Base
from system_evaluation import test_sistema
from Models.melhorado.Autoencoder import Autoencoder, EarlyStopping
import pickle


In [2]:
first_stage = torch.load(os.path.join("Models","melhorado","checkpoint.pt"))
with open(os.path.join("Models","melhorado", "scaler_ae.p"), 'rb') as f:
    scaler = pickle.load(f)

first_stage.scaler = scaler


In [3]:
with open("Models\melhorado\SecondStage.p","rb") as f:
    second_stage = pickle.load(f)
second_stage

Pipeline(steps=[('scaler',
                 QuantileTransformer(output_distribution='normal',
                                     subsample=100000)),
                ('floresta',
                 RandomForestClassifier(max_features=0.1751204590963604,
                                        max_samples=0.9034128710297624,
                                        n_estimators=200, random_state=42))])

In [4]:
class SistemaHierarquico(Sistema_Hierarquico_Base):
        def __init__(self,anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u):
            super().__init__(anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u)

        def detect_anomaly_proba(self, X):
            # vetor com classe de maior probabilidade de cada instancia
            # pegam so a probabilidade dos elementos serem 1
            y_score = self.anomaly_detector.score_samples(X, scale_data= True)

            # normaliza para ficar entre 0 e 1 como uma probabilidade
            y_score = (y_score - y_score.min())/(y_score.max() -y_score.min())

            return y_score

        def classify_proba(self,X):
            # matriz, cada linha representa uma instancia que pde ser maligna,
            # cada coluna representa uma classe
            # cada elemento da matriz representa uma probabilidade
            return self.classifier.predict_proba(X)

In [5]:
test_full_model   = pd.read_csv("data\\test_full_model.csv")
test_set          = pd.read_csv("data\\test_dataset.csv")

print("test_dataset")
print(test_set.info())
print("-"*100, "\n\n")

print("test_full_model")
print(test_full_model.info())
print("-"*100, "\n\n")



test_dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56860 entries, 0 to 56859
Data columns (total 68 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                56860 non-null  int64  
 1   Flow Duration             56860 non-null  float64
 2   Total Fwd Packets         56860 non-null  float64
 3   Total Backward Packets    56860 non-null  float64
 4   Fwd Packets Length Total  56860 non-null  float64
 5   Bwd Packets Length Total  56860 non-null  float64
 6   Fwd Packet Length Max     56860 non-null  float64
 7   Fwd Packet Length Min     56860 non-null  float64
 8   Fwd Packet Length Mean    56860 non-null  float64
 9   Fwd Packet Length Std     56860 non-null  float64
 10  Bwd Packet Length Max     56860 non-null  float64
 11  Bwd Packet Length Min     56860 non-null  float64
 12  Bwd Packet Length Mean    56860 non-null  float64
 13  Bwd Packet Length Std     56860 non-null  float6

In [6]:
nome_anomalias = list(second_stage.classes_)
nome_labels    = ["Unknown"] + nome_anomalias + ["BENIGN"]
nome_labels

['Unknown',
 '(D)DOS',
 'Botnet',
 'Brute Force',
 'Port Scan',
 'Web-Attack',
 'BENIGN']

In [7]:
test_dataset_list = [
                        test_full_model,
                        test_set           
                    ]
test_dataset_names = ["test_full_model","test_set"]

In [31]:
sh = SistemaHierarquico(first_stage,second_stage, 5, 0.00043491917, .85, .08) 

In [32]:
test_sistema(sh,test_dataset_list, test_dataset_names, nome_labels)

test_full_model
              precision    recall  f1-score   support

      (D)DOS       0.32      0.99      0.49      1076
      BENIGN       1.00      0.88      0.94    100000
      Botnet       0.24      1.00      0.39      1040
 Brute Force       0.46      1.00      0.62      1100
   Port Scan       0.96      1.00      0.98      1066
     Unknown       0.00      0.00      0.00         0
  Web-Attack       0.42      0.99      0.59      1038

    accuracy                           0.89    105320
   macro avg       0.49      0.84      0.57    105320
weighted avg       0.97      0.89      0.92    105320

---------------------------------------------------------------------------------------------------- 


test_set
              precision    recall  f1-score   support

      (D)DOS       0.33      0.98      0.49       584
      BENIGN       1.00      0.91      0.95     54387
      Botnet       0.21      0.99      0.35       477
 Brute Force       0.58      0.98      0.73       584
   

In [30]:
time_spent_each_stage = sh.get_execution_time_lists()

print("tempos em exec (segundos)")
pd.DataFrame([np.round(time_spent,2) for  time_spent in time_spent_each_stage], columns= test_dataset_names, index= ["Stage1", "Stage2", "Stage3", "Sistema"])
    

tempos em exec (segundos)


,test_full_model,test_set
Stage1,0.18,0.08
Stage2,1.96,0.78
Stage3,0.00,0.00
Sistema,2.14,0.86
